In [1]:
# Requirments.txt
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statistics as stats

In [3]:
path = r"C:\Users\ameya\GitHub\EnergyDemandForecasting_TimeSeriesAnalysis\Dataset\historic_demand_2009_2022.csv"
df_energy = pd.read_csv(path)

In [4]:
df_energy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 243936 entries, 0 to 243935
Data columns (total 21 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   Unnamed: 0                 243936 non-null  int64  
 1   settlement_date            243936 non-null  object 
 2   settlement_period          243936 non-null  int64  
 3   nd                         243936 non-null  int64  
 4   tsd                        243936 non-null  int64  
 5   england_wales_demand       243936 non-null  int64  
 6   embedded_wind_generation   243936 non-null  int64  
 7   embedded_wind_capacity     243936 non-null  int64  
 8   embedded_solar_generation  243936 non-null  int64  
 9   embedded_solar_capacity    243936 non-null  int64  
 10  non_bm_stor                243936 non-null  int64  
 11  pump_storage_pumping       243936 non-null  int64  
 12  ifa_flow                   243936 non-null  int64  
 13  ifa2_flow                  24

In [6]:
df_energy.head()

,Unnamed: 0,settlement_date,settlement_period,nd,tsd,england_wales_demand,embedded_wind_generation,embedded_wind_capacity,embedded_solar_generation,embedded_solar_capacity,...,pump_storage_pumping,ifa_flow,ifa2_flow,britned_flow,moyle_flow,east_west_flow,nemo_flow,nsl_flow,eleclink_flow,is_holiday
0,0,2009-01-01,1,37910,38704,33939,54,1403,0,0,...,33,2002,0,0,-161,0,0,NaN,NaN,1
1,1,2009-01-01,2,38047,38964,34072,53,1403,0,0,...,157,2002,0,0,-160,0,0,NaN,NaN,1
2,2,2009-01-01,3,37380,38651,33615,53,1403,0,0,...,511,2002,0,0,-160,0,0,NaN,NaN,1
3,3,2009-01-01,4,36426,37775,32526,50,1403,0,0,...,589,1772,0,0,-160,0,0,NaN,NaN,1
4,4,2009-01-01,5,35687,37298,31877,50,1403,0,0,...,851,1753,0,0,-160,0,0,NaN,NaN,1


In [13]:
df_energy.isnull().any()

Unnamed: 0                   False
settlement_date              False
settlement_period            False
nd                           False
tsd                          False
england_wales_demand         False
embedded_wind_generation     False
embedded_wind_capacity       False
embedded_solar_generation    False
embedded_solar_capacity      False
non_bm_stor                  False
pump_storage_pumping         False
ifa_flow                     False
ifa2_flow                    False
britned_flow                 False
moyle_flow                   False
east_west_flow               False
nemo_flow                    False
nsl_flow                      True
eleclink_flow                 True
is_holiday                   False
dtype: bool